## Equal weight S&P 500 Index Fund

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math 

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

### Acquiring an API token

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
#USE IEX CLOUD API
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable//stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json() 
#print(data)

### Parsing data from API

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

In [6]:
#Adding stock data to pandas dataframe
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dt = pd.DataFrame(columns = my_columns)
final_dt

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
final_dt.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
        index = my_columns
    ),
    ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,134.7,2245571168674,N/A


In [8]:
#Looping through the tickers in the list of stocks
final_dt = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable//stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dt = final_dt.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
            index = my_columns
        ),
        ignore_index = True
    )

In [9]:
#Using batch API calls - helper function
def chunks(lst, n):
    """Yield successive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [10]:
#Using batch API calls
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dt = pd.DataFrame(columns = my_columns)

for ss in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={ss}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #parse data for every stock
    for symbol in ss.split(','):
        final_dt = final_dt.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
                index= my_columns
            ),
            ignore_index=True
        )
final_dt

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,122.170,37231929741,N/A
1,AAL,15.384,9169532689,N/A
2,AAP,160.560,11030893447,N/A
3,AAPL,131.440,2204210537393,N/A
4,ABBV,110.220,191796599103,N/A
...,...,...,...,...
500,YUM,106.870,32590642034,N/A
501,ZBH,159.730,33156310996,N/A
502,ZBRA,380.620,20830456544,N/A
503,ZION,43.910,7279993040,N/A


In [15]:
portfolio_size = input('Enter the value of your portfolio: ')
try:
    val = float(portfolio_size)
except ValueError:
    print("Please enter an integer")
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)
    

Enter the value of your portfolio: 1000000


In [16]:
position_size = val/len(final_dt.index)
for i in range(0, len(final_dt.index)):
    final_dt.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dt.loc[i, 'Stock Price'])
final_dt

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,122.170,37231929741,16
1,AAL,15.384,9169532689,128
2,AAP,160.560,11030893447,12
3,AAPL,131.440,2204210537393,15
4,ABBV,110.220,191796599103,17
...,...,...,...,...
500,YUM,106.870,32590642034,18
501,ZBH,159.730,33156310996,12
502,ZBRA,380.620,20830456544,5
503,ZION,43.910,7279993040,45


In [25]:
#Writing results to excel file
writer = pd.ExcelWriter('recommended_trades.xlsx', engine= 'xlsxwriter')
final_dt.to_excel(writer, 'Recommended Trades', index = False)

In [26]:
#Creating formats for xlsx
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)


In [27]:
#Applying formats to columns (long way)
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()

In [28]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [29]:
writer.save()